In [1]:
using PyCall, ProgressBars

# Load in the Models

In [10]:
# Imports for calling model
pushfirst!(pyimport("sys")."path", "/home/smkatz/Documents/Classes/CS230/CS230_DAA/src")
xplane_ctrl = pyimport("util")
model1 = xplane_ctrl.load_model("models/uniform_v1.pt")
model2 = xplane_ctrl.load_model("models/uniform_v2.pt")
model3 = xplane_ctrl.load_model("models/uniform_v3.pt");

# Get Detections

In [3]:
detects_train_1 = zeros(9500)
bbs_train_1 = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v1/train/images/$(i).jpg"
    detects_train_1[i+1], bbs = xplane_ctrl.bb_from_file(model1, filename)
    push!(bbs_train_1, bbs)
    i % 200 == 0 ? println(i) : nothing
end

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400


In [4]:
detects_valid_1 = zeros(500)
bbs_valid_1 = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v1/valid/images/$(i).jpg"
    detects_valid_1[i-9499], bbs = xplane_ctrl.bb_from_file(model1, filename)
    push!(bbs_valid_1, bbs)
    i % 200 == 0 ? println(i) : nothing
end

9600
9800


In [5]:
detects_train_2 = zeros(9500)
bbs_train_2 = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v2/train/images/$(i).jpg"
    detects_train_2[i+1], bbs = xplane_ctrl.bb_from_file(model2, filename)
    push!(bbs_train_2, bbs)
    #i % 200 == 0 ? println(i) : nothing
end

detects_valid_2 = zeros(500)
bbs_valid_2 = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v2/valid/images/$(i).jpg"
    detects_valid_2[i-9499], bbs = xplane_ctrl.bb_from_file(model2, filename)
    push!(bbs_valid_2, bbs)
    #i % 200 == 0 ? println(i) : nothing
end

detects_train_3 = zeros(9500)
bbs_train_3 = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v3/train/images/$(i).jpg"
    detects_train_3[i+1], bbs = xplane_ctrl.bb_from_file(model3, filename)
    push!(bbs_train_3, bbs)
    #i % 200 == 0 ? println(i) : nothing
end

detects_valid_3 = zeros(500)
bbs_valid_3 = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v3/valid/images/$(i).jpg"
    detects_valid_3[i-9499], bbs = xplane_ctrl.bb_from_file(model3, filename)
    push!(bbs_valid_3, bbs)
    #i % 200 == 0 ? println(i) : nothing
end

# Get Detections Loaded

In [1]:
using BSON

In [3]:
res = BSON.load("/scratch/smkatz/cs230_detects_data.bson")
detects_train_1 = res[:detects_train_1]
detects_train_2 = res[:detects_train_2]
detects_train_3 = res[:detects_train_3]
detects_valid_1 = res[:detects_valid_1]
detects_valid_2 = res[:detects_valid_2]
detects_valid_3 = res[:detects_valid_3]

res = BSON.load("/scratch/smkatz/cs230_bbs_data.bson")
bbs_train_1 = res[:bbs_train_1]
bbs_train_2 = res[:bbs_train_2]
bbs_train_3 = res[:bbs_train_3]
bbs_valid_1 = res[:bbs_valid_1]
bbs_valid_2 = res[:bbs_valid_2]
bbs_valid_3 = res[:bbs_valid_3];

# Get Labels

In [4]:
using DelimitedFiles

In [5]:
bbs_train_1_labels = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v1/train/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_train_1_labels, res[2:5])
end

In [6]:
bbs_valid_1_labels = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v1/valid/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_valid_1_labels, res[2:5])
end

In [7]:
bbs_train_2_labels = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v2/train/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_train_2_labels, res[2:5])
end

bbs_valid_2_labels = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v2/valid/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_valid_2_labels, res[2:5])
end

bbs_train_3_labels = []
for i = 0:9499
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v3/train/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_train_3_labels, res[2:5])
end

bbs_valid_3_labels = []
for i = 9500:9999
    filename = "/scratch/smkatz/yolo_data/yolo_format/uniform_v3/valid/labels/$(i).txt"
    res = readdlm(filename)
    push!(bbs_valid_3_labels, res[2:5])
end

In [8]:
function to_pixel(label)
    xp = label[1] * 1920
    yp = label[2] * 1056
    wp = label[3] * 1920
    hp = label[4] * 1056
    return [xp, yp, wp, hp]
end

to_pixel (generic function with 1 method)

In [9]:
bbs_valid_1[1]

1×4 Matrix{Float64}:
 588.0  343.0  38.0  14.0

In [10]:
to_pixel(bbs_valid_1_labels[1])'

1×4 adjoint(::Vector{Float64}) with eltype Float64:
 600.876  349.946  38.2234  14.0152

# Get IoU

In [11]:
using LazySets

In [12]:
function iou(bb1, bb2)
    h1 = Hyperrectangle(bb1[1:2], bb1[3:4])
    h2 = Hyperrectangle(bb2[1:2], bb2[3:4])

    intersection = h1 ∩ h2
    union = h1 ∪ h2

    ia = area(intersection)
    ua = area(h2) + area(h1) - ia

    return ia / ua
end

iou (generic function with 1 method)

In [15]:
function get_iou(detects, bbs_pred, bbs_labels)
    ious = []
    for i = 1:length(detects)
        if detects[i] > 0
            res = zeros(size(bbs_pred[i], 1))
            for j = 1:size(bbs_pred[i], 1)
                res[j] = iou(bbs_pred[i][j, :], to_pixel(bbs_labels[i]))
            end
            push!(ious, maximum(res))
        end
    end
    return ious
end

get_iou (generic function with 1 method)

In [16]:
ious_train_1 = get_iou(detects_train_1, bbs_train_1, bbs_train_1_labels)
ious_valid_1 = get_iou(detects_valid_1, bbs_valid_1, bbs_valid_1_labels)
ious_train_2 = get_iou(detects_train_2, bbs_train_2, bbs_train_2_labels)
ious_valid_2 = get_iou(detects_valid_2, bbs_valid_2, bbs_valid_2_labels)
ious_train_3 = get_iou(detects_train_3, bbs_train_3, bbs_train_3_labels)
ious_valid_3 = get_iou(detects_valid_3, bbs_valid_3, bbs_valid_3_labels);

In [59]:
using Statistics

In [60]:
println("Mean IoU Train 1: ", mean(ious_train_1))
println("Mean IoU Train 2: ", mean(ious_train_2))
println("Mean IoU Train 3: ", mean(ious_train_3))
println("Mean IoU Valid 1: ", mean(ious_valid_1))
println("Mean IoU Valid 2: ", mean(ious_valid_2))
println("Mean IoU Valid 3: ", mean(ious_valid_3))

Mean IoU Train 1: 0.3640849550191995
Mean IoU Train 2: 0.36470560306866173
Mean IoU Train 3: 0.36513088640998176
Mean IoU Valid 1: 0.37586723454782234
Mean IoU Valid 2: 0.36110893797971966
Mean IoU Valid 3: 0.3608479622838801


# Above vs. Below Horizon

In [24]:
ypixels1 = [to_pixel(bbs_train_1_labels[i])[2] for i = 1:length(bbs_train_1_labels)];

In [25]:
ypixels1_detect = ypixels1[findall(detects_train_1 .> 0)];

In [16]:
# using BSON
# using BSON: @save

In [17]:
# @save "/scratch/smkatz/cs230_detects_data.bson" detects_train_1 detects_train_2 detects_train_3 detects_valid_1 detects_valid_2 detects_valid_3
# @save "/scratch/smkatz/cs230_bbs_data.bson" bbs_train_1 bbs_train_2 bbs_train_3 bbs_valid_1 bbs_valid_2 bbs_valid_3

In [18]:
using PGFPlots

In [39]:
include("/home/smkatz/Documents/Util/support_code.jl");

In [51]:
ax = Axis(Plots.Histogram(ypixels1, bins=20, density=true, style="gray, fill=gray!50, opacity=0.5", legendentry="All Images"), 
            xmin=0)
push!(ax, Plots.Histogram(ypixels1_detect, bins=20, density=true, style="pastelBlue, fill=pastelBlue!50, opacity=0.5", legendentry="Detected Images"))
ax.xlabel="Vertical Pixel Location"
ax.ylabel="Density"
ax.xmax = 1056
ax
PGFPlots.save("/home/smkatz/Documents/RiskSensitivePerception/detect_compare.tex", ax, include_preamble=false)